In [42]:
import numpy as np

In [43]:
def evaluate(x):
    px = np.array([0,  50, 100, 300, 400, 700, 799], dtype=int)
    py = np.array([0, 100,   0,   0,  25,   0,  50], dtype=int)
    xx = np.arange(0, 800)
    yy = np.interp(xx, px, py)
    return -yy[x]  # negative altitude, becase we are minimizing (to be consistent with other obj. functions)

In [44]:
def update_dist(best):
    x = np.arange(800)
    dist = np.interp(x, x[best<0], np.abs(best[best<0]))
    # normalize
    dist = (1/np.sum(dist))*dist
    #print(dist)
    return dist

In [45]:
def sample_dist(dist):
    ratios = np.round(10000*dist)
    rng = np.random.rand(100,800)
    large_dist = np.array([dist,]*100)
    large_x = np.array([np.arange(800),]*100)

    pick = rng < large_dist
    #print(pick.shape)
    #print(large_x.shape)
    #print(np.sum(pick))
    sample = large_x[pick].reshape(np.sum(pick))
    shuffle = np.random.permutation(np.sum(pick))
    return sample[shuffle[:10]]

In [46]:
def eda():
    x = np.random.randint(0, 800, size=(11,10))
    y = np.zeros(shape=(10,10))
    dist = (1/800)*np.ones(shape=(10,800))
    best = np.zeros(800)
    last_change = np.zeros(800)
    for i in range(10):
        y[i,:] = evaluate(x[i,:])
        ind = [k for k in x[i,:]]
        pos = y[i,:] < best[x[i,:]]
        last_change[x[i,pos]] = i*np.ones(1,np.sum(pos))
        best[x[i,:]] = np.minimum(best[x[i,:]], y[i,:])

        dist[i,:] = update_dist(best)
        #print(np.sum(dist[i,:] > np.zeros(shape=(1,800))))
        x[i+1,:] = sample_dist(dist[i,:])
    final_x = np.argmin(best)
    final_y = np.min(best)
    return final_y, last_change[final_x]

In [47]:
def run_multiple(n, fun):
    best_val = np.zeros(n)
    iter_no = np.zeros(n)
    for i in range(n):
        best_val[i], iter_no[i] = fun()
    
    target_reached = np.sum(best_val == -100)
    average_score = np.mean(best_val)
    average_iters = np.mean(iter_no)

    print(f"Targer value -100 reached in {target_reached} runs, which is {target_reached*100/1000} %.")
    print(f"Average score is {average_score}.")
    print(f"Average number of iterations to converge is {average_iters}.")

    return target_reached, average_score, average_iters



In [48]:
ex_target, ex_score, ex_iters = run_multiple(1000,eda)

Targer value -100 reached in 368 runs, which is 36.8 %.
Average score is -96.54981818181818.
Average number of iterations to converge is 5.317.


In [49]:
def pso(swarm_size, obj_fct):
    x = np.random.randint((10,10))
    v = np.random.randint((10,10))
    iter_no = int(100/swarm_size)
    for i in range(iter_no):
        y = evaluate(x[i,:])
        v
    